In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv

hv.extension("bokeh")
from datetime import datetime, timedelta

In [ ]:
%load_ext autoreload
%autoreload 2
import inventory
from inventory import File

In [ ]:
inventory.connect_db("paulsson.full.db")

# Date correspondence

In [ ]:
files = File.select().where(File.type == "nd2").order_by(File.size.desc()).limit(10)

In [ ]:
md = files[0].metadata

In [ ]:
md.keys()

In [ ]:
print(md["image_text_info"]["SLxImageTextInfo"]["TextInfoItem_5"])

In [ ]:
md["image_metadata"]

In [ ]:
md["acquisition_time"]

In [ ]:
import jdutil

In [ ]:
import pytz

In [ ]:
md["image_metadata_sequence"]["SLxPictureMetadata"]["dTimeMSec"] / (60 * 1000)

In [ ]:
md["image_metadata_sequence"]["SLxPictureMetadata"]  # ['dTimeStart']

In [ ]:
dtimemsec = md["image_metadata_sequence"]["SLxPictureMetadata"]["dTimeMSec"]
dtimemsec

In [ ]:
dtimemsec_dt = timedelta(milliseconds=dtimemsec)
dtimemsec_dt

In [ ]:
acquisition_time_nyc + dtimemsec_dt

In [ ]:
dtimeabsolute = jdutil.jd_to_datetime(
    md["image_metadata_sequence"]["SLxPictureMetadata"]["dTimeAbsolute"]
)
dtimeabsolute

In [ ]:
nyc = pytz.timezone("America/New_York")

In [ ]:
pytz.UTC.localize(dtimeabsolute).astimezone(nyc)

In [ ]:
x = float(md["acquisition_time"]["variant"]["no_name"]["@value"])

In [ ]:
acquisition_time = jdutil.jd_to_datetime(x)
acquisition_time

In [ ]:
acquisition_time_nyc = pytz.UTC.localize(acquisition_time).astimezone(nyc)
acquisition_time_nyc

In [ ]:
mtime = nyc.localize(datetime.fromtimestamp(files[0].mtime))
ctime = nyc.localize(datetime.fromtimestamp(files[0].ctime))

In [ ]:
dt = mtime - acquisition_time_nyc

In [ ]:
dt.total_seconds() / 60 / 60

In [ ]:
md["acquisition_times"][-100:] / (60 * 1000) / 60  # / (1000*60)#*60)

In [ ]:
files[0].size / 10**12

# Old

In [ ]:
files = (
    File.select(File.mtime, File.size).where(File.type == "nd2").order_by(File.mtime)
)

In [ ]:
len(files)

In [ ]:
files[2].mtime

In [ ]:
mtimes, sizes = zip(*[(f.mtime, f.size) for f in files])

In [ ]:
sizes = np.array(sizes)

In [ ]:
cumsizes = np.cumsum(sizes)

In [ ]:
hv.Curve((mtimes, cumsizes))

In [ ]:
hv.Histogram?

In [ ]:
hv.Histogram(*np.histogram(sizes, 100)).opts(plot={"logy": False})

In [ ]:
sizes = np.array([dd["size"] for dd in d])
sizes[::-1].sort()

In [ ]:
plt.hist([dd["size"] for dd in d], bins=100, log=True)

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(np.cumsum(sizes))